In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import scipy.stats as ss

import sklearn.model_selection as ms 

In [2]:
# load your data (don't touch, just run)

data = []
f = open('covtype.data','r')
while(1):
    line = f.readline()
    if  len(line) < 100:
        print(line)
    
    if len(line) == 0: break
    data.append(np.array([float(k) for k in line.split(',')]))
    if len(data) % 100000 == 0:
        print(len(data))
        
f.close
data = np.vstack(data)
N = data.shape[0]
idx = np.random.permutation(N)


X_test = data[:N//5,:]
X_train = data[N//5:,:]
y_test = X_test[:,-1]
y_train = X_train[:,-1]
X_test = X_test[:,:-1]
X_train = X_train[:,:-1]


sio.savemat('covtype.mat',{'X_train':X_train,'X_test':X_test,'y_train':y_train,'y_test':y_test})

data = sio.loadmat('covtype.mat')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train'][0]
y_test = data['y_test'][0]

y_idx_train = [np.where(np.equal(y_train,k))[0] for k in np.unique(y_train)]

for i in range(len(y_idx_train)):
    y_idx = y_idx_train[i]
    y_idx_train[i] = y_idx[np.random.choice(len(y_idx),len(y_idx)//1000+1,replace=False)]
    
y_idx_train = np.hstack(y_idx_train)
y_idx_train = np.random.permutation(y_idx_train)

X_train = X_train[y_idx_train,:]
y_train = y_train[y_idx_train]

sio.savemat('covtype_reduced.mat',{'X_train':X_train,'X_test':X_test,'y_train':y_train,'y_test':y_test})


100000
200000
300000
400000
500000



In [3]:

data = sio.loadmat('covtype_reduced.mat')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train'][0]
y_test = data['y_test'][0]

print(np.unique(y_train), np.unique(y_test))

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


[1. 2. 3. 4. 5. 6. 7.] [1. 2. 3. 4. 5. 6. 7.]
(468, 54) (116202, 54) (468,) (116202,)


In [4]:
from math import log,e

def entropy(label):
    unique, count = np.unique(label, return_counts=True, axis=0)
    probability = count/len(label)
    entroph = np.sum(probability*np.log2(probability) * -1)
    return entroph

def cond_entropy(label,split):
    #n.c concatenates stacked 1-D arrays as colums into a 2-D array
    jointDist = np.c_[label,split]
    entro = entropy(jointDist)
    return entro - entropy(split)

random_sequences = sio.loadmat('random_sequences.mat')

s1 = random_sequences['s1'][0]
s2 = random_sequences['s2'][0]

print('entropy = ', entropy(s1))
print('conditional entropy = ', cond_entropy(s1,s2))
    

entropy =  3.3141823231610834
conditional entropy =  3.3029598816135173


In [5]:
# def find_best_split(x,y):
#     best_feat = 0
#     splitval = 0.
#     set1 = range(int(len(y)/2))
#     set2 = range(int(len(y)/2),len(y))
#     return best_feat, splitval, set1, set2
def find_best_split(x,y):
    #x train a matrix and y train a vector
    maxVar = float("-inf")
    set1 = range(int(len(y)/2))
    set2 = range(int(len(y)/2), len(y))
    best_feat = 0
    splitval = 0.

    for c in range(0, len(x[0])):
        for r in np.unique(x):
            tempset1 = np.where(x[:,c] < r)
            tempset2 = np.where(x[:,c] >= r)
            
            weight1 = len(tempset1) / len(y)
            weight2 = len(tempset2) / len(y)

            informationGain = entropy(y) - (weight1*entropy(y[tempset1])) - (weight2*entropy(y[tempset2]))

            if informationGain > maxVar:
                maxVar = max(maxVar, informationGain)
                set1 = tempset1
                set2 = tempset2
                best_feat = c
                splitval = r
    
    return best_feat, splitval, set1, set2

best_feat, splitval, set1, set2 = find_best_split(X_train, y_train)
y_new = y_train * 0
y_new[set1] = 1
print('information gained in first step', (entropy(y_train) - cond_entropy(y_train,y_new)) * 10)

information gained in first step 0.5881140851070632


In [6]:
def purity(y):
    return ss.mode(y)[1]/len(y+0.)
    
class Node:
    def __init__(self,  sample_idx, nodeid,  is_leaf = True):
        self.is_leaf = is_leaf
        self.id = nodeid
        self.sample_idx = sample_idx
        self.children = []        
        
    def visit_node(self, x):
        if self.is_leaf:
            return self.label
        """ Fill me in """
        if x[self.splitfeat] < splitval:
            return self.children[0].visit_node(x)
        else:
            return self.children[1].visit_node(x)

    def add_split_details(self, splitfeat, splitval)  :
        self.splitfeat = splitfeat
        self.splitval = splitval
    
        
class Tree:
    def __init__(self, x,y):
        m = len(y)
        self.x = x
        self.y = y
        self.maxid = -1
        self.root = self.construct_node(np.array(range(m)))
        self.leaves = [self.root]
        
    def print_tree(self):
        print('printing tree...')
        def print_node(parent, node):
            print(node.id, end='')
            
            if parent is not None:
                print(', parent ', parent.id,end='')
            else:
                print(', ROOT',end='')
                
            print(', label ', node.label, end='')
            if node.is_leaf: 
                print(', LEAF, ', 'nsamples %d, purity %.2f' %(len(node.sample_idx), purity(self.y[node.sample_idx])))
            else:
                print(', NONLEAF, split %d, val %.2f' % (node.splitfeat, node.splitval))
            if not node.is_leaf:
                for ch in node.children:
                    print_node(node, ch)
        print_node(None, self.root)
        
        
    def construct_node(self, sample_idx):
        node = Node(sample_idx, self.maxid + 1,  True)

        node.label = ss.mode(self.y[sample_idx])[0][0] # fill me in
        
        node.entropy = entropy(self.y[sample_idx])
        node.num_mistakes = np.sum(np.not_equal(node.label, self.y[sample_idx]))
        self.maxid += 1
        return node
        
    def report_train_err(self):
        total_mistakes = 0
        for leaf in self.leaves:
            total_mistakes += leaf.num_mistakes
        return total_mistakes / (len(self.y)+0.)
        
    
    
    def predict(self,x):
        return self.root.visit_node(x)


    

In [7]:
def get_test_err(tree):
    # get test error
    num_test_mistakes = 0
    for k in range(len(y_test)):
        x,y = X_test[k,:],y_test[k]
        if y != tree.predict(x):
            num_test_mistakes += 1
    return num_test_mistakes / (len(y_test)+0.)



tree = Tree(X_train,y_train)
tree.print_tree()
print('current train err:', tree.report_train_err())
print('current test err:', get_test_err(tree))


# my first split
best_feat, splitval, set1, set2 = find_best_split(X_train, y_train)
 
left_child = tree.construct_node(set1)
right_child = tree.construct_node(set2)
tree.root.is_leaf = False
tree.leaves.pop(tree.leaves.index(tree.root))
tree.root.add_split_details(splitfeat = best_feat, splitval = splitval)


tree.root.children = [left_child, right_child]
tree.leaves.extend(tree.root.children)
tree.print_tree()

for i in range(0, 24):
    treecurrLeaf = tree.leaves.pop(0)
    tempy_train = y_train[treecurrLeaf.sample_idx]
    tempx_train = X_train[treecurrLeaf.sample_idx,:]
    
    if purity(tempy_train) != 1:
        best_feat, splitval, set1, set2 = find_best_split(tempx_train, tempy_train)
        left_child = tree.construct_node(set1)
        right_child = tree.construct_node(set2)
        treecurrLeaf.is_leaf = False
        treecurrLeaf.children.add_split_details(splitfeat = best_feat, splitval = splitval)

        treecurrLeaf.children = [left_child, right_child]
        tree.leaves.extend(treecurrLeaf.children)
tree.print_tree()
print('twenty five train err:', tree.report_train_err())
print('twenty five test err:', get_test_err(tree))


printing tree...
0, ROOT, label  2.0, LEAF,  nsamples 468, purity 0.44
current train err: 0.5641025641025641
current test err: 0.3138069912738163
printing tree...
0, ROOT, label  2.0, NONLEAF, split 0, val 2234.00
1, parent  0, label  3.0, LEAF,  nsamples 1, purity 1.00
2, parent  0, label  2.0, LEAF,  nsamples 1, purity 0.44


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed